In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd
from pathlib import Path
import os
import random

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine, Sphere, Rgba, CspaceFreeRegion
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)




pendulum_world = FindResourceOrThrow("drake/C_Iris_Examples/assets/double_pendulum_world.urdf")

models = []
models.append(parser.AddModelFromFile(pendulum_world))

plant.Finalize()
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(1)

# #compute limits in t-space
limits_q = [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]
limits_s = [Ratfk.ComputeTValue(np.array(q), q_star) for q in limits_q]
do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram


DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/package.xml' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/double_pendulum_example.runfiles/drake/package.xml' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/C_Iris_Examples/assets/double_pendulum_world.urdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/double_pendulum_example.runfiles/drake/C_Iris_Examples/assets/double_pendulum_world.urdf' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:Fi

In [6]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)

id_pairs_of_interest = [(gid1, gid2) for i, gid1 in enumerate(gids) for gid2 in gids[i+1:] ]
print(id_pairs_of_interest)
visualizer.collision_pairs_of_interest = id_pairs_of_interest


[(<GeometryId value=42>, <GeometryId value=28>), (<GeometryId value=42>, <GeometryId value=25>), (<GeometryId value=28>, <GeometryId value=25>)]


In [7]:
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))


## Set up the sliders so we can move the plant around manually

In [9]:
sliders = []

for i, q in enumerate(plant.GetPositionLowerLimits()):
    sliders.append(widgets.FloatSlider(min=q,
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.array([0,0])
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.57, min=-1.57)

FloatSlider(value=0.0, description='q1', max=1.57, min=-1.57)

DEBUG:drake:Meshcat connection opened from 0000:0000:0000:0000:0000:0000:0000:0001
DEBUG:drake:Meshcat connection closed from 0000:0000:0000:0000:0000:0000:0000:0001
DEBUG:drake:Meshcat connection opened from 0000:0000:0000:0000:0000:0000:0000:0001


In [19]:
s0 = np.array([0,0])
s1_safe = np.array([-0.25, 0.9])
s1_unsafe = np.array([1,1])

def plot_edge(s0, s1, s0_name = "s0", s1_name = "s1"):
    visualizer.meshcat2.SetObject(f"/{s0_name}",
                                 Sphere(0.05),
                                 Rgba(0.0, 1.0, 0, 1))
    s0_3d = np.array([s0[0],s0[1],0])
    s1_3d = np.array([s1[0],s1[1],0])
    print(s0_3d)
    print(s1_3d)
    visualizer.meshcat2.SetTransform(f"/{s0_name}",
                                  RigidTransform(RotationMatrix(),
                                                 s0_3d))
    
    visualizer.meshcat2.SetObject(f"/{s1_name}",
                                 Sphere(0.05),
                                 Rgba(1.0, 0.0, 0, 1))
    visualizer.meshcat2.SetTransform(f"/{s1_name}",
                                   RigidTransform(RotationMatrix(),
                                                 s1_3d))
    visualizer.meshcat2.SetLine(f"/edge_{s0_name}_{s1_name}", 
                                np.hstack([s0_3d[:,np.newaxis], s1_3d[:, np.newaxis]]),
                                line_width = 2, rgba = Rgba(0.0, 0.0, 0.0, 1))
plot_edge(s0,s1_safe, s1_name = "s1_safe")
plot_edge(s0,s1_unsafe, s1_name = "s1_unsafe")
plot_edge(limits_s[0],limits_s[1], s0_name = "joint limit low", s1_name = "joint limit high")

[0 0 0]
[-0.25  0.9   0.  ]
[0 0 0]
[1 1 0]
[-0.99920399 -0.99920399  0.        ]
[0.99920399 0.99920399 0.        ]


In [ ]:
limits_s[1]-eps

In [ ]:
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kAffine, 
                                np.zeros(1),
                                set(), VerificationOption()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

In [ ]:

safe = line_certifier.CertifyTangentConfigurationSpaceLine(s0,s1_safe)
print(safe)
safe = line_certifier.CertifyTangentConfigurationSpaceLine(s0,s1_unsafe)
print(safe)
